In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()

spark = SparkSession.builder.appName("Practice-itv021172-2 ").config('spark.ui.port','0').config('spark.sql.warehouse.dir',f"/user/itv021172/warehouse").enableHiveSupport().master('yarn').getOrCreate()

In [2]:
schema = '''loan_id string,member_id string,
loan_amount float,funded_amount float,loann_term_month string,interest_rate float,installment float,
issue_date string,loan_status string,loan_purpose string,
title string
'''

In [3]:
df = spark.read.format('csv').option('header',True).schema(schema).load('/public/trendytech/lendingclubproject/raw/loans_data_csv/part-00000-30d34559-cada-4e5a-91c1-381f533f4c78-c000.csv')

In [4]:
from pyspark.sql.functions import *

In [5]:
loan = df.withColumn('Ingest_date',current_timestamp())

In [6]:
loandf=loan.dropna(subset=['funded_amount','loan_amount','loann_term_month','interest_rate','installment','issue_date','loan_status','loan_purpose'])

In [9]:
freshloan = loandf.withColumn('loann_term_month',(regexp_replace(col('loann_term_month'),' months','') \
.cast('int')/12) \
.cast('int')) \
.withColumnRenamed('loann_term_month','loan_term_year')

In [17]:
reason = ['debt_consolidation','credit_card','home_improvement','major_purchase','medical','small_business','car','vacation','moving','house','wedding','renewable_energy','educational']

In [18]:
loan_withReason = freshloan.withColumn('loan_purpose',when(col('loan_purpose').isin(reason),col('loan_purpose')).otherwise('other'))

In [22]:
loan_withReason

loan_id,member_id,loan_amount,funded_amount,loan_term_year,interest_rate,installment,issue_date,loan_status,loan_purpose,title,Ingest_date
56633077,b59d80da191f5b573...,3000.0,3000.0,3,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-08-25 11:25:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-08-25 11:25:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-08-25 11:25:...
56463188,e12aefc548f750777...,11200.0,11200.0,5,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2025-08-25 11:25:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-08-25 11:25:...
56663266,1c4329e5f17697127...,20000.0,20000.0,5,13.33,458.45,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-08-25 11:25:...
56483027,5026c86ad983175eb...,10000.0,10000.0,3,12.69,335.45,Aug-2015,Fully Paid,other,Other,2025-08-25 11:25:...
56613385,9847d8c1e9d0b2084...,23400.0,23400.0,5,19.19,609.46,Aug-2015,Current,small_business,Business,2025-08-25 11:25:...
56643620,8340dbe1adea41fb4...,16000.0,16000.0,3,5.32,481.84,Jul-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-08-25 11:25:...
56533114,d4de0de3ab7d79ad4...,25450.0,25450.0,3,27.31,1043.24,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-08-25 11:25:...
